### Regularization

Regularization merupakan sebuah teknik untuk mencegah terjadinya ovefitting, dengan cara memodifikasi hidden layer agar tidak terlalu cocok dengan training data dan membuat model deep learning lebih general. Biasanya regularization mengurangi akurasi training data, namun meningkatkan akurasi testing data

Misal pada kasus klasifikasi di bawah

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import os
os.environ["KERAS_BACKEND"] = "plaidml.bridge.keras"

Deskripsi dataset: This is a dataset that describes sonar chirp returns bouncing off different services. The 60 input variables are the strength of the returns at different angles. It is a binary classification problem that requires a model to differentiate rocks from metal cylinders.

In [2]:
data= pd.read_csv('sonar.all-data', header=None)

data

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


Mengecek nilai NaN

In [3]:
[idx for idx, null in enumerate(data.isnull().sum()) if null>0]

[]

Mengecek jika terdapat kolom objek dan jika ada maka print nilai uniknya

In [4]:
[f'Col: {col}, values: {data[col].unique()}' for col in data if data[col].dtypes==object]

["Col: 60, values: ['R' 'M']"]

Encode kolom target

In [5]:
data[60].replace({'R':1, 'M':0}, inplace=True)

data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,1


Train test split

In [6]:
x= data.drop(60, axis=1)
y= data[60]

x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.3)

x_train.shape, y_train.shape

((145, 60), (145,))

## Membangun Model

Membuat model tanpa regularization

In [7]:
model= keras.Sequential()

model.add(keras.layers.Dense(64, input_dim= 60, activation='relu'))
model.add(keras.layers.Dense(32, activation= 'relu'))
model.add(keras.layers.Dense(16, activation= 'relu'))
model.add(keras.layers.Dense(1, activation= 'sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100)

Epoch 1/100
5/5 [==============================] - 0s 976us/step - loss: 0.7091 - accuracy: 0.4828
Epoch 2/100
5/5 [==============================] - 0s 998us/step - loss: 0.6869 - accuracy: 0.6138
Epoch 3/100
5/5 [==============================] - 0s 2ms/step - loss: 0.6763 - accuracy: 0.6138
Epoch 4/100
5/5 [==============================] - 0s 998us/step - loss: 0.6667 - accuracy: 0.6345
Epoch 5/100
5/5 [==============================] - 0s 998us/step - loss: 0.6581 - accuracy: 0.6552
Epoch 6/100
5/5 [==============================] - 0s 999us/step - loss: 0.6501 - accuracy: 0.6828
Epoch 7/100
5/5 [==============================] - 0s 2ms/step - loss: 0.6398 - accuracy: 0.6897
Epoch 8/100
5/5 [==============================] - 0s 999us/step - loss: 0.6286 - accuracy: 0.7034
Epoch 9/100
5/5 [==============================] - 0s 2ms/step - loss: 0.6145 - accuracy: 0.7034
Epoch 10/100
5/5 [==============================] - 0s 999us/step - loss: 0.6003 - accuracy: 0.7241
Epoch 11/100
5/

5/5 [==============================] - 0s 999us/step - loss: 0.0611 - accuracy: 0.9931
Epoch 84/100
5/5 [==============================] - 0s 1ms/step - loss: 0.0591 - accuracy: 0.9931
Epoch 85/100
5/5 [==============================] - 0s 0s/step - loss: 0.0623 - accuracy: 0.9931
Epoch 86/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0598 - accuracy: 0.9931
Epoch 87/100
5/5 [==============================] - 0s 0s/step - loss: 0.0525 - accuracy: 0.9931
Epoch 88/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0602 - accuracy: 0.9931
Epoch 89/100
5/5 [==============================] - 0s 0s/step - loss: 0.0493 - accuracy: 0.9931
Epoch 90/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0499 - accuracy: 0.9931
Epoch 91/100
5/5 [==============================] - 0s 0s/step - loss: 0.0469 - accuracy: 0.9931
Epoch 92/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0486 - accuracy: 0.9931
Epoch 93/100
5/5 [=================

In [8]:
model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 0s/step - loss: 0.3755 - accuracy: 0.8571


[0.375479131937027, 0.8571428656578064]

### L1 dan L2 Regularization

L1 dan L2 Regularization merupakan teknik regularisasi yang menekan agar weight dari suatu layer tidak terlalu tinggi dan mengurangi kompleksitas model. Ada rumusnya, parameternya adalah lambda (biasanya 0.01), yang jika terlalu tinggi maka semakin menekan weight, dan jika terlalu rendah maka biasanya tetap terjadi overfitting. Perbedaan l1 dan l2 adalah l1 lebih menekan weight mendekati 0 daripada l2

Membuat model dengan l1 regularization

In [9]:
model_l1= keras.Sequential([
    keras.layers.Dense(64, input_dim= 60, activation= 'relu'),
    # Di sini kita menambahkan regularizers pada layer ini dengan lambda 0.005, cara menentukan lamba dengan coba-coba    
    keras.layers.Dense(32, activation= 'relu', kernel_regularizer= keras.regularizers.l1(0.005)),
    keras.layers.Dense(16, activation= 'relu'),
    keras.layers.Dense(1, activation= 'sigmoid')
])

model_l1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_l1.fit(x_train, y_train, epochs=100)

Epoch 1/100
5/5 [==============================] - 0s 0s/step - loss: 1.9459 - accuracy: 0.5586
Epoch 2/100
5/5 [==============================] - 0s 0s/step - loss: 1.8888 - accuracy: 0.6069
Epoch 3/100
5/5 [==============================] - 0s 0s/step - loss: 1.8339 - accuracy: 0.6207
Epoch 4/100
5/5 [==============================] - 0s 0s/step - loss: 1.7821 - accuracy: 0.6000
Epoch 5/100
5/5 [==============================] - 0s 0s/step - loss: 1.7294 - accuracy: 0.6759
Epoch 6/100
5/5 [==============================] - 0s 0s/step - loss: 1.6792 - accuracy: 0.6966
Epoch 7/100
5/5 [==============================] - 0s 0s/step - loss: 1.6293 - accuracy: 0.7655
Epoch 8/100
5/5 [==============================] - 0s 4ms/step - loss: 1.5803 - accuracy: 0.7379
Epoch 9/100
5/5 [==============================] - 0s 0s/step - loss: 1.5324 - accuracy: 0.6966
Epoch 10/100
5/5 [==============================] - 0s 4ms/step - loss: 1.4827 - accuracy: 0.7310
Epoch 11/100
5/5 [===================

5/5 [==============================] - 0s 4ms/step - loss: 0.4214 - accuracy: 0.8828
Epoch 86/100
5/5 [==============================] - 0s 0s/step - loss: 0.3923 - accuracy: 0.9034
Epoch 87/100
5/5 [==============================] - 0s 4ms/step - loss: 0.3944 - accuracy: 0.8828
Epoch 88/100
5/5 [==============================] - 0s 0s/step - loss: 0.3894 - accuracy: 0.8966
Epoch 89/100
5/5 [==============================] - 0s 0s/step - loss: 0.3932 - accuracy: 0.8828
Epoch 90/100
5/5 [==============================] - 0s 0s/step - loss: 0.3816 - accuracy: 0.8966
Epoch 91/100
5/5 [==============================] - 0s 0s/step - loss: 0.3853 - accuracy: 0.8966
Epoch 92/100
5/5 [==============================] - 0s 0s/step - loss: 0.3802 - accuracy: 0.9034
Epoch 93/100
5/5 [==============================] - 0s 0s/step - loss: 0.3776 - accuracy: 0.9103
Epoch 94/100
5/5 [==============================] - 0s 0s/step - loss: 0.3762 - accuracy: 0.9103
Epoch 95/100
5/5 [=======================

In [10]:
model_l1.evaluate(x_test, y_test)

2/2 [==============================] - 0s 0s/step - loss: 0.5288 - accuracy: 0.7778


[0.5287560224533081, 0.7777777910232544]

Model dengan l2 regularization

In [11]:
model_l2= keras.Sequential([
    # Di sini kita melakukan regularization pada 2 layer
    keras.layers.Dense(64, input_dim= 60, activation= 'relu', kernel_regularizer= keras.regularizers.l2(0.01)),
    keras.layers.Dense(32, activation= 'relu', kernel_regularizer= keras.regularizers.l2(0.01)),
    keras.layers.Dense(16, activation= 'relu'),
    keras.layers.Dense(1, activation= 'sigmoid')
])

model_l2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_l2.fit(x_train, y_train, epochs=100)

Epoch 1/100
5/5 [==============================] - 0s 0s/step - loss: 1.7170 - accuracy: 0.4345
Epoch 2/100
5/5 [==============================] - 0s 3ms/step - loss: 1.6525 - accuracy: 0.5793
Epoch 3/100
5/5 [==============================] - 0s 0s/step - loss: 1.5928 - accuracy: 0.5931
Epoch 4/100
5/5 [==============================] - 0s 4ms/step - loss: 1.5371 - accuracy: 0.6207
Epoch 5/100
5/5 [==============================] - 0s 0s/step - loss: 1.4851 - accuracy: 0.6207
Epoch 6/100
5/5 [==============================] - 0s 0s/step - loss: 1.4352 - accuracy: 0.6207
Epoch 7/100
5/5 [==============================] - 0s 0s/step - loss: 1.3880 - accuracy: 0.6207
Epoch 8/100
5/5 [==============================] - 0s 0s/step - loss: 1.3436 - accuracy: 0.6276
Epoch 9/100
5/5 [==============================] - 0s 4ms/step - loss: 1.3009 - accuracy: 0.6483
Epoch 10/100
5/5 [==============================] - 0s 0s/step - loss: 1.2594 - accuracy: 0.6414
Epoch 11/100
5/5 [==================

5/5 [==============================] - 0s 0s/step - loss: 0.3663 - accuracy: 0.9586
Epoch 86/100
5/5 [==============================] - 0s 4ms/step - loss: 0.3634 - accuracy: 0.9448
Epoch 87/100
5/5 [==============================] - 0s 0s/step - loss: 0.3628 - accuracy: 0.9655
Epoch 88/100
5/5 [==============================] - 0s 4ms/step - loss: 0.3616 - accuracy: 0.9448
Epoch 89/100
5/5 [==============================] - 0s 0s/step - loss: 0.3521 - accuracy: 0.9586
Epoch 90/100
5/5 [==============================] - 0s 4ms/step - loss: 0.3530 - accuracy: 0.9517
Epoch 91/100
5/5 [==============================] - 0s 0s/step - loss: 0.3491 - accuracy: 0.9586
Epoch 92/100
5/5 [==============================] - 0s 4ms/step - loss: 0.3477 - accuracy: 0.9517
Epoch 93/100
5/5 [==============================] - 0s 0s/step - loss: 0.3421 - accuracy: 0.9655
Epoch 94/100
5/5 [==============================] - 0s 4ms/step - loss: 0.3401 - accuracy: 0.9655
Epoch 95/100
5/5 [====================

In [12]:
model_l2.evaluate(x_test, y_test)

2/2 [==============================] - 0s 7ms/step - loss: 0.5920 - accuracy: 0.7460


[0.592026948928833, 0.7460317611694336]

### Dropout Regularization

Dropout regularization merupakan teknik regularisasi dengan cara menghilangkan beberapa node (biasanya 50% atau 0.5) dari suatu hidden layer secara acak pada setiap epoch. Parameternya adalah berapa persentase node yang mau dihilangkan secara acak, misal 0.5 maka akan dihilangkan 50%.

Dropout regularization efektif jika digunakan pada model neural network dengan stuktur yang besar untuk memperkenalkan lebih banyak keacakan.

Model dengan dropout regularization

In [13]:
model_dropout= keras.Sequential([
    keras.layers.Dense(64, input_dim= 60, activation= 'relu'),
    keras.layers.Dense(32, activation= 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(16, activation= 'relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(1, activation= 'sigmoid')
])

model_dropout.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_dropout.fit(x_train, y_train, epochs=100)

Epoch 1/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6965 - accuracy: 0.4897
Epoch 2/100
5/5 [==============================] - 0s 0s/step - loss: 0.6905 - accuracy: 0.5241
Epoch 3/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6774 - accuracy: 0.5655
Epoch 4/100
5/5 [==============================] - 0s 0s/step - loss: 0.6733 - accuracy: 0.5931
Epoch 5/100
5/5 [==============================] - 0s 0s/step - loss: 0.6782 - accuracy: 0.5724
Epoch 6/100
5/5 [==============================] - 0s 0s/step - loss: 0.6748 - accuracy: 0.5103
Epoch 7/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6716 - accuracy: 0.5655
Epoch 8/100
5/5 [==============================] - 0s 0s/step - loss: 0.6699 - accuracy: 0.5655
Epoch 9/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6662 - accuracy: 0.5103
Epoch 10/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6581 - accuracy: 0.5862
Epoch 11/100
5/5 [================

5/5 [==============================] - 0s 0s/step - loss: 0.1683 - accuracy: 0.9448
Epoch 85/100
5/5 [==============================] - 0s 4ms/step - loss: 0.1437 - accuracy: 0.9655
Epoch 86/100
5/5 [==============================] - 0s 0s/step - loss: 0.1252 - accuracy: 0.9655
Epoch 87/100
5/5 [==============================] - 0s 0s/step - loss: 0.1425 - accuracy: 0.9448
Epoch 88/100
5/5 [==============================] - ETA: 0s - loss: 0.1737 - accuracy: 0.93 - 0s 0s/step - loss: 0.1512 - accuracy: 0.9448
Epoch 89/100
5/5 [==============================] - 0s 0s/step - loss: 0.1356 - accuracy: 0.9517
Epoch 90/100
5/5 [==============================] - 0s 4ms/step - loss: 0.1346 - accuracy: 0.9517
Epoch 91/100
5/5 [==============================] - 0s 0s/step - loss: 0.1403 - accuracy: 0.9517
Epoch 92/100
5/5 [==============================] - 0s 4ms/step - loss: 0.1218 - accuracy: 0.9586
Epoch 93/100
5/5 [==============================] - 0s 1ms/step - loss: 0.1394 - accuracy: 0.96

In [14]:
model_dropout.evaluate(x_test, y_test)

2/2 [==============================] - 0s 0s/step - loss: 0.3348 - accuracy: 0.8889


[0.3347786068916321, 0.8888888955116272]